# Japanese fake news classification

We're going to attempt to classify this dataset. This is a dataset featuring Japanese news articles, from which a part
real, some are half fake, and some are entirely fake.

0: Original article
1: Partially fake
2: Completely fake

Source: [Japanese fakenews dataset](https://www.kaggle.com/tanreinama/japanese-fakenews-dataset)

## Inspection

In [7]:
import pandas as pd

df = pd.read_csv("fakenews.csv")

df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
isfake,13040.0,1.075613,0.796950,0.0,0.0,1.0,2.00,2.0
nchar_real,13040.0,261.029371,288.257753,0.0,0.0,215.0,407.00,4447.0
nchar_fake,13040.0,328.273160,354.673240,0.0,0.0,255.0,489.25,2582.0


In [8]:
print(f'Rows/Columns: {df.shape}')
print(f"Class distribution is: \n{df['isfake'].value_counts()}\n")
print(df.isnull().sum())

Rows/Columns: (13040, 5)
Class distribution is: 
1    4684
2    4671
0    3685
Name: isfake, dtype: int64

id            0
context       0
isfake        0
nchar_real    0
nchar_fake    0
dtype: int64


From what we can see the dataset does not contain null values, and it's class distribution is mostly even.

Although mostly even, not perfectly even, so while my first thought was accuracy as a metric, i've decided to use the
ROC curve instead.

In [9]:
print(df['context'].head())

0    朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コン...
1    11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置...
2    産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2...
3    開催地のリオデジャネイロ市に対して、大会期間中のリオデジャネイロオリンピックに関する公式発表...
4    毎日新聞・時事通信によると、2006年2月13日には、グッドウィル・グッゲンハイム・アン・ハ...
Name: context, dtype: object


## Preprocessing

We can mostly follow the standard NLP cleaning methods, albeit with a catch, it's a character based language, so the
traditional packages such as NLTK won't work. That's why we need to find substitutes for the cleaning methods. Let's go
over them:

I've decided to go with [SudachiPy](https://pypi.org/project/SudachiPy/) for tokenization,
[JapaneseStemmer](https://github.com/asherperkins11/Japanese-stemmer) (you'll have to copy the file locally) for stemming and
[stopwords-ja](https://github.com/stopwords-iso/stopwords-ja/blob/master/stopwords-ja.json) (link to json file) for stopword removal.

### Removing features

Everything besides the actual text and the label class is just noise, so we'll take those out.

In [10]:
labels = df.pop('isfake')
data = df.pop('context')

### Tokenization

We're going to tokenize our sentences. This we'll do with Sudachi, a morphological analyzer. The short version of
what that means is that we're splitting our sentences up into pieces, aka tokens. We need a specialized analyzer for
this task, because you can't just randomly split it up by character, because then you'd lose context. After tokenization
is complete we can do the rest of the preprocessing steps, which we usually apply individually per character.

### Special characters

We need to also remove some special characters that will be of no use to the classifier, think of stuff such as commas
and question marks, etc. We do this via a regex filter. I spent some time looking for the exactly right one online,
since I wanted to keep all japanese characters, english text, and numbers.

### Stemming

Stemming implies reducing the form of a word to its stem. E.g. 食べている -->　食べる, した　--> する. This is useful to
reduce the amount of variants of a word which mostly mean the exact same thing (from the perspective of the model)

### Stopwords

Stopword removal implies the removal of certain words which appear in an excessive frequency in the language. Common
examples for English are "the", "and" and so forth. These are useful to remove because they are so often used that they
lose meaning.

In [17]:
def sample(data, limit=3):
    """
    Quick util method to display samples of the sentences
    :param data: collection
    :param limit: how much samples to show
    """
    for x in range(0, limit, 1):
        print(f"{data[x]}\n\n")

sample(data)

朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された。2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた。朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの。朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる。日本郵政公社は、公正妥当な判決とのコメントを出した。一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている。


11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置することから、人身売買により、環境問題に加え、環境保護にも関心が向けられた。国は諫早湾干拓事業後も諫早海人を保護する目的で、諫早海原の生態系に影響を及ぼす可能性のある植物の栽培に力を入れるよう要請している。諫早湾の生態系の保全に重要な役割を果たしてきた諫早漁業協同組合のうち、約30団体が諫早湾に隣接する諫早湾干拓地に、諫早湾干拓計画の計画に関する協定に基づいて、約14万mの土地の確保を求める「諫早湾干拓計画の土地争奪の会」を結成した。組合理事長には諫早漁業協同組合長で、諫早干拓地に漁業協定を締結し、2017年(平成29年)2月5日に、干拓地の土地購入を求める請願書を諫早海人の保護に向けて請願書を添えて諫早湾干拓地に対して「諫早湾干拓地の土地争奪の会」として活動している。


産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2006年から2年連続で実施したことがある。ま

## Conclusion

Package wise, we still have a long way to go. The Japanese equivalents of the NLP packages are more underveloped,
less reliable, and generally slower too. This can be mostly attributed to the language being difficult, but it's
something you have to keep in mind while working with Japanese text. Maybe in a few years the tools will be at a good
level.


## References

[Preprocessing Methods and Tools in Modelling Japanese for Text Classification](https://www.researchgate.net/publication/335337209_Preprocessing_Methods_and_Tools_in_Modelling_Japanese_for_Text_Classification)
Paper detailing tools for Japanese NLP analysis. A few years old, but still relevant as of current date.

[nlp-recipes-ja](https://github.com/upura/nlp-recipes-ja)
Github repository containing a ton of samples for Japanese text analysis in Python.
